# Fairness

Da wir in der Vorlesung inzwischen bei anwendungsbezogeneren Themen angekommen sind, wollen wir die verbliebenen Übungen nutzen, um Euch etwas freier an Aufgabenstellungen arbeiten zu lassen. Die Konzepte der Vorlesungen sollen dabei als Grundgedanken bei der Auswahl und Implementierung der Algorithmen einfließen. Heute soll es, passend zur Vorlesung, um Fairness gehen.

Seit vielen Jahren sind Risk Assessment Tools gebräuchlich in den Gerichten der USA. Sie sollen die Richter bei der Entscheidung über das Strafmaß unterstützen und als neutrale Komponente (ohne Bias) zum Prozess beitragen. Dabei kommt es regelmäßig vor, dass diese Tools maßgeblich die Entscheidung zu Haftdauer und Strafmaß beeinflussen und manchmal sogar die Vorschläge der Jury überstimmen.

Die Effektivität und Genauigkeit, sowie die Anwendung dieser Tools ist stark umstritten. Da es in dieser Übung um Fairness gehen soll, gehen wir erst einmal davon aus, dass die Modelle prinzipiell funktionieren und wir wollen uns vor allem auf die Auswirkungen auf einzelne Gruppen konzentrieren.

Das Risk Assessment Tool um das es in der Übung gehen soll ist das *Correctional Offender Management Profiling for Alternative Sanctions (COMPAS)* vom Hersteller Northpointe. Dieses war 2016 groß in den Schlagzeilen als ProPublica, ein Netzwerk investigativer Journalisten, die Ergebnisse der Risikobewertung für ein County in Florida für die Jahre 2012/2013 unter dem Gesichtspunkt *Racial Bias*, insbesondere schwarze/weiße Amerikaner, untersuchte und zu dem Schluss kam, dass es einen starken Bias gegen schwarze Amerikaner gebe.

COMPAS liefert eine Reihe Scores, welche auf 137 Features (Fragen) basieren, die entweder aus Fragebögen der Angeklagten oder der kriminellen Vorgeschichte hervorgehen. Dabei wird *Race* explizit nicht abgefragt.

In der Übung betrachten wir den *Decile Score* für "Risk of Recidivism" und wie gut dieser eine Rückfälligkeit innerhalb zwei Jahren nach Urteil vorhersagt, sprich ob der Angeklagte eine weitere Straftat im Zeitraum von zwei Jahren begeht (gekennzeichnet mit `two_year_recid`). Mit diesem Label wurde das COMPAS Modell auch trainiert. Der Score kann diskrete Werte von 1 bis 10 annehmen, wobei 1-4 als niedriges, 5-7 als mittleres und 8-10 als hohes Risiko der Rückfälligkeit eingestuft werden.

Außerdem gibt es auch den Decile Score für "Risk of Violence", welcher entsprechend das Risiko für Gewaltstraftaten angeben soll.

In der Übung soll nun also dieses Modell (COMPAS) unter den Fairnesskriterien, welche wir in der Vorlesung behandelt haben, untersucht werden.


**Hier noch ein paar Links zu der ProPublica Debatte:**

*Disclaimer: Es ist womöglich sinnvoll zuerst die Übung (unvoreingenommen) zu machen und selbst herauszufinden was einem auffällt. Die Übung soll keine politische Stellungname bilden, sondern soll nur die in der Vorlesung dargestellten Kriterien an einem realen und interessanten Beispiel anwenden.*


ProPublica Analyse: https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm


Außerdem sehr interessanter Vortrag zum Thema Fairness, welcher in Teilen auch COMPAS behandelt:

https://www.youtube.com/watch?v=p5yY2MyTJXA

Kritik an der Analyse von ProPublica gab es unter anderem von [Flores et al]: "It might be that the existing justice system is biased agains poor minorities due to a wide variety of reasons (including economic factors, policing patterns, prosecutorial behavior, and judicial biases), and therefore, regardless of the degree of bias, risk assessment tools informed by objective data can help *reduce* racial bias from its current level. It would be a shame if policymakers mistakenly thought that risk assessment tools were somehow worse than the status quo. Because we are at a time in history where there appears to be bipartisan political support for criminal justice reform, one poorly executed study that makes such absolute claims of bias should not go unchallenged. The gravity of this study's erroneous conclusions is exacerbated by the large-market outlet in which it was published."


## Aufgaben:
Ist der Score noch zu retten? Um das zu klären, gehen wir wie in der Vorlesung vor:

1. Frage:  Schaue zuerst die Daten an. Was sind hier geschützte Attribute?

2. Frage: Betrachte den Decile Score als den Output des Models (vgl. α in VL) und plotte die Verteilung der Scores in Abhängikeit der 2 Jahres Rückfälligkeit getrennt nach den geschützten Attributen. Prüfe ob der COMPAS Score calibriert ist bezüglich "Caucasian"/"African American". Kannst du Aussagen zur Sufficiency treffen?

2. Angenommen wir möchten vorhersagen, ob eine Person ein "hohes" Risiko oder "kein hohes" Risiko hat rückfällig zu werden. Ist das Kriterium der Independence für die betroffenen Gruppen erfüllt?

3. Wie müssen wir den Schwellwert auf den Score für unsere Entscheidung legen um die Vorhersagegenauigkeit zur Rückfälligkeit für Menschen der Gruppen "Caucasian" zu maximieren?
Wende das Prinzip der Independence an um den Schwellwert für die Gruppe "African-American" zu ermitteln.

4. Kann das Kriterium der Separation für diese beiden Gruppen erfüllt werden? Unter welchen Vorrausetzungen ist dies der Fall? Nutze die ROC-Kurve dazu. Wie sehen die Thresholds jeweils aus?

5. Zurück zur Sufficiency: Ist Sufficency für identische Schwellwerte für beide Gruppen erfüllt? Was passiert, wenn wir Separation fordern und entsprechende Schwellwerte auf den Score wählen? Lassen sich alle drei Kriterien gleichzeitig erfüllen?


*Tipp: Du kannst alle Aufgaben durch geschickte Plots beantworten. Außerdem sind unterhalb dieser Aufgabenstellung noch ein paar Hilfestellungen/Erinnerungen verfübar.*


Zuerst importieren wir wieder die Pakete, welche wir für die Übung benötigen. Seaborn ist eine erweiterte API für Matplotlib und ermöglicht einfache und hübsche Plots.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


Dann laden wir die Daten von ProPublica

In [ ]:
url = 'https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv'
df = pd.read_csv(url)
df.info()
df

Du kannst die Daten wie ProPublica filtern.

In [ ]:
#ProPublica wendet zusätzlich folgenden Filter an (den man aber auch ignorieren kann)
df = df[(df.days_b_screening_arrest<=30)&(df.days_b_screening_arrest>=-30)]
df.shape

(6172, 53)

In [ ]:
# 1. Geschützte Attribute?
# Plot der Verteilung der Scores:


**Zur Erinnerung hier nochmal ein paar Definitionen:**

- $\tilde{r}$ :  Als rückfallgefährdet eingestuft
- $r$ : wird tatsächlich rückfällig

**Unabhängigkeit**:

Wahrscheinlichkeit als rückfallgefährdet eingestuft zu werden soll für beide Gruppen gleich sein $P(\tilde{r}=1|\text{Gruppe}=\text{"Caucasian"}) \overset{!}{=} P(\tilde{r}=1|\text{Gruppe}=\text{"African-American"})$

**Separation**

TPR und FPR sollen gleich sein, das bedeutet das Modell soll gleich gut darin sein Menschen als rückfallgefährdet vorherzusagen unabhängig der Gruppenzugehörigkeit.

$P(\tilde{r}=1|r=1,\text{Gruppe}=\text{"Caucasian"}) \overset{!}{=} P(\tilde{r}=1|r=1,\text{Gruppe}=\text{"African-American"})$
$P(\tilde{r}=1|r=0,\text{Gruppe}=\text{"Caucasian"}) \overset{!}{=} P(\tilde{r}=1|r=0,\text{Gruppe}=\text{"African-American"})$

**Sufficiency**

Der Anteil an als gefährdet vorhergesagter Menschen die tatsächlich rückfällig werden soll für beide Gruppen identisch sein.

Rückfällige möglichst gut erkennen <--> Anteil an fälschlicherweise als Rückfallgefährdeten möglichst gering halten

![](https://www.researchgate.net/profile/Mauno-Vihinen/publication/230614354/figure/fig4/AS:216471646019585@1428622270943/Contingency-matrix-and-measures-calculated-based-on-it-2x2-contigency-table-for.png)